# OpenStreetMap Data Case Study

### Mapa da Area a ser analisada


Manhattan (8398124) in New York Citt, NY, United States of America

- [https://www.openstreetmap.org/relation/8398124](https://www.openstreetmap.org/relation/8398124)

Query realizada no site abaixo para buscar os dados do quadrante que será utilizado para análise:
- [http://overpass-api.de/query_form.html](http://overpass-api.de/query_form.html)

Com a query:
Latitude mínima: 40.6827 e máxima: 40.8808
Longitude mínima: -74.0486 e máxima: -73.9043

(
node(40.6827000, -74.0486000, 40.8808000, -73.9043000);
<;
);out meta;

Este mapa se refere ao quadrante referente a Manhanttan em New York City, e também areas ao redor devido ao quadrante.
O motivo de eu ter pego este local e não minha cidade "São Paulo", pois é um lugar que gostei muito de visitar, e desejo retornar para umas férias mais longas, e até mesmo viver lá, o segundo motivo pela escolha é, que este dataset irá me auxiliar no enriquecimento de dados em outro projeto que estou trabalhando, para um projeto no MBA para análise de dados dos taxis de NYC.

### Ambiente utilizado para desenvolvimento da análise:


- Máquina: Macbook pro 2017 core I7 com 16GB de Ram, com sistema Operacional (macOS Hith Sierra 10.13.5 (17F77))

- Anaconda, criei o environment py36
- Python 3.6, jupyter, VS Code 1.25.1, pymongo 3.4.0
- MongoDb no Docker: [https://hub.docker.com/r/library/mongo/](https://hub.docker.com/r/library/mongo/)

#### Estrutura do projeto:


-- amostra_arquivo.py (responsável por gerar o arquivo de amostra com k=10 e k=20)

-- data_wrangling.py  (responsável por realizar a auditoria e limpeza de dados, assim como ler o XML (map_sample.osm) e gerar o json para ser importado no mongodb (map.osm.json)

-- data_insert_in_mongodb.py (responsável por ler o json criado (map.osm.json) e inserir os dados no mongodb)

-- data (diretório onde ficará os artefatos a serem processados e os logs)
+ ------ map.osm (arquivo contendo o OSM da área do mapa que foi buscado no item (Mapa da Area a ser analisada))
+ ------ map.osm-auditing.log (arquivo com logs de auditoria do processamento)
+ ------ map.osm.json ( arquivo json que será inserido no mongodb)
+ ------ map.osm.json-error-to-insert-json.log (arquivo de log com itens que tiveram erro no processamento ao inserir no mongoDB)

Obs.1: Devido os artefatos serem muito grandes, estou disponibilizando o map.osm (map.osm.zip) e map.osm.json (map.osm.json.zip) no meu drive remoto (oneDrive):

[https://1drv.ms/f/s!Au31I8svXayr6hS42acMhYrBXnzC](https://1drv.ms/f/s!Au31I8svXayr6hS42acMhYrBXnzC)

Obs.2: Os arquivos de logs da auditoria com respostas das auditorias map.osm-auditing.log e do processamento do json para o mongoDB  map.osm.json-error-to-insert-json.log na pasta data

Obs.: As querys foram executadas diretamente no mongoDB


## "Problems Encountered in the Map" e Validações de dados e chaves para auditoria

Após baixar o arquivo do mapa que irei utilizar, rodei o script amostra_arquivo.py com k = 10 para gerar um arquivo menor com amostragem dos dados de exemplo para iniciar o trabalho de análise, com isso pude ter uma visão geral dos tipos de chaves e tags do dataset.

Os scripts de auditoria e limpeza dos dados está no arquivo data_wrangling.py, para análise de auditoria e processamento do arquivo json, que foi gerado para iniciar a analise de dados no mongodb, criei a função main:

```python 
    def main(filename):
      json_list = []

      tags_auditing = {}
      tag_k_auditing = {}
      tag_k_v_yes_no_auditing = set()
      postal_code = set()
      street_address = {}
      for event, element in ET.iterparse(filename):
        street_address = audit_street_name(street_address, element)
        postal_code = audit_postal_code(postal_code, element)
        audit_count_tag_attribute_k_with_v_yes_no(tag_k_v_yes_no_auditing, element)
        tag_k_auditing = audit_count_tag_attribute_k(tag_k_auditing, element)

        if element.tag in MAIN_TAGS:
          tags_auditing = audit_tags_subtags(tags_auditing, element)

          json_list.append(process_json(element, tag_k_v_yes_no_auditing))

      # You do not need to change this file
      file_out = "{0}.json".format(filename)
      with codecs.open(file_out, "w") as fo:
        fo.write(json.dumps(json_list))
```

#### 1- Auditoria inicial de tags e atributos para avaliar consistências de dados

Iniciei com a análise de contabilização de tags e chaves de atributos para verificar se existia algum desbalanceamento de estrutura para cada tag do xml e seus atributos, verifiquei que não existe inconsistências.



#### 2- Auditoria de tags e subtags e seus atributos para avaliar consistências de dados

Após a análise acima, conforme a documentação dos dados, as tags principais são node, relation e way, desta forma fiz uma análise das possíveis tags abaixo de cada


Com base nesta análise foi possível identificar que para as tags principais pode-se ter as subtags:
Node [ tag ] 
Relation [ member, tag ]
Way [ nd, tag ]

#### 3- Auditoria das chaves `"k"` das tags de subelementos ` "tag"` para avaliar consistência e ajustes do json gerado

Com o fragmento de codigo abaixo iterando pelos elementos, do tipo 'tag', foi verificado a chave *k* e contando a quantidade de ocorrencias das chaves, e dessa forma será possível melhorar a estrutura e verificar os itens que deverão ter maior necessidade para auditar e limpar os dados:

```Python
def audit_count_tag_attribute_k(tag_k_auditing, element):
  
  if element.tag == 'tag':
    v = 1
    k = element.attrib['k']
    if k in tag_k_auditing:
      v = tag_k_auditing[k] + 1
    tag_k_auditing[k] = v
  return tag_k_auditing
```

Resposta:


Problemas encontrados no mapeamento acima: 
- a chave k="Phone" deverá ficar junto da tag do json phone do objeto
- O o calor da chave 'addr:zip' deverá ser mapeado no mesma tag da chave 'addr:postcode'

Conforme verificado na documentação do wiki, foi identificado alguns padrões de map features, sendo assim irei criar a tag primary_map_feature : {key : value, ...} para facilitar em categorização de possíveis tags, conforme mapeado acima.
[https://wiki.openstreetmap.org/wiki/Map_Features](https://wiki.openstreetmap.org/wiki/Map_Features)

Mapeado no código como uma lista:
PRIMARY_FEATURES = [
'aerialway','aeroway','amenity','barrier',
'boundary','building','craft','emergency',
'geological','highway','cycleway','busway',
'sidewalk','historic','landuse','leisure',
'man_made','military','natural','office',
'place','power','line','public_transport',
'railway','route','shop','sport',
'tourism','waterway'
]

#### 4- Auditoria de tags e estruturação do json

O mapeamento do json será definido conforme definicoes abaixo, para facilitar o mapeamento e análise:

- Todas as chaves iniciadas com 'addr' serão mapeadas no json da chave 'address'
- Todas as chaves iniciadas com 'building:*' serão agrupadas no json na chave 'building'
- Todas as chaves iniciadas com 'cityracks.*' serão agrupadas no json na chave 'cityracks'
- Todas as chaves iniciadas com 'crossing*' serão agrupadas no json na chave 'crossing'
- Todas as chaves que iniciam com 'gnis:*' serão agrupadas no json na chave 'gnis'
- A chave 'name' ficará na chave 'name', as demais chaves que iniciarem com 'name' ou 'old_name' irão ser agrupadas na chave do json 'names'
- Todas as chaves iniciadas com 'tiger:*' serão agrupadas no json na chave 'tiger'
- Todas as chaves que contenham o valor como `"yes"` / `"no"` deverão ser agrupadas na chave do json 'restrictions'
- Todas as chaves que possuem a palavra `conditional` serão agrupadas na chave do json como 'restrictions'
- As chaves 'opening_hours' serão consideradas como restrictions com o valor yes na chave principal


#### 5- Auditoria de dados de tags com valores condicionais, e possíveis regras

Após a auditoria de valores com yes / no como esperado as tags com keys condicionais que possuem divergências:

'motor_vehicle:conditional': {'no @ (Mo-Fr 10:00-08:00; Sa,Su)',
                               'no @ (Su 08:00-18:00)'},
                               
``` xml
 <tag k="access:conditional" v="yes @ (axles=2 AND weight&lt;40 st); yes @ (axles=3 AND weight&lt;60 st); yes @ (axles=4 AND weight&lt;70 st); yes @ (axles&gt;=5 AND weight&lt;80 st)"/>

 <tag k="motor_vehicle:conditional" v="no @ (Mo-Fr 10:00-08:00; Sa,Su)"/>

 <tag k="opening_hours" v="Mo,Sa,Su,PH 09:00-17:00; Tu-Fr 08:00-20:00"/>
 <tag k="opening_hours" v="Mo-Fr 11:00-23:00; Sa,Su 10:00-23:00"/>
 <tag k="opening_hours" v="Mo-Fr 8:00-16:00, 17:00-23:00; Sa 9:00-16:00, 17:00-23:00; Su 9:00-17:00"/>
````
                               
As seguintes tags com keys possuem o valor com yes / no e possuem outros valores devido a isso no momento que for gerado o json ficarão dentro da chave 'restrictions':

Obs.: Devido a chave de condicional poder possuir mais de uma condição conforme listado acima, no momento da limpeza dos dados irei realizar o seguinte ajuste no momento do condicional:

- Modificar '`&lt;`' por ' < '
- Modificar '`&gt;`' por ' > '
- Fazer split pela string '); ' para separar as possiveis condicoes
- Fazer split do valor por ' @ ' para separar o atributo inicial (yes / no / ...) do valor da condição
- Caso o valor possuir mais de 1 caracter @
entre outras regras

Criado funções para normalizar os dados de condicionais, para retornar o dado de condicional conforme abaixo, gerado dados para testar algumas condições:

```text
"yes @ (axles=2 AND weight&lt;40 st); yes @ (axles=3 AND weight&lt;60 st); yes @ (axles=4 AND weight&lt;70 st); yes @ (axles&gt;=5 AND weight&lt;80 st); no_left_turn @ (Mo-Fr 06:00-10:00,15:00-19:00); no_left_turn @ (Mo-Sa 07:00- 20:00); permissive @ (Mo-Fr 07:00-22:00; SH off);yes @ (Mo-Th 09:00-17:00; Fr 09:00-18:00; Sa 10:00-14:00)"
```


```JSON
{'no_left_turn': [{'friday': {'06:00-10:00', '15:00-19:00'},
                   'monday': {'06:00-10:00', '15:00-19:00'},
                   'thursday': {'06:00-10:00', '15:00-19:00'},
                   'tuesday': {'06:00-10:00', '15:00-19:00'},
                   'wednesday': {'06:00-10:00', '15:00-19:00'}},
                  {'friday': {'07:00-20:00'},
                   'monday': {'07:00-20:00'},
                   'saturday': {'07:00-20:00'},
                   'thursday': {'07:00-20:00'},
                   'tuesday': {'07:00-20:00'},
                   'wednesday': {'07:00-20:00'}}],
 'permissive': [{' sh off': None,
                 'friday': {'07:00-22:00'},
                 'monday': {'07:00-22:00'},
                 'thursday': {'07:00-22:00'},
                 'tuesday': {'07:00-22:00'},
                 'wednesday': {'07:00-22:00'}}],
 'yes': [{'axles=2 and weight < 40 st': None},
         {'axles=3 and weight < 60 st': None},
         {'axles=4 and weight < 70 st': None},
         {'axles >= 5 and weight < 80 st': None},
         {'friday': {'09:00-18:00'},
          'monday': {'09:00-17:00'},
          'saturday': {'10:00-14:00'},
          'thursday': {'09:00-17:00'},
          'tuesday': {'09:00-17:00'},
          'wednesday': {'09:00-17:00'}}]}
```

Verificado com o codigo:

```python

def audit_count_tag_attribute_k_with_v_yes_no(tag_k_v_yes_no_auditing, element):
  if element.tag == 'tag':
    v = element.attrib['v']
    k = element.attrib['k']
    if v == 'yes' or v == 'no' or v.startswith('yes ') or v.startswith('no ') or 'conditional' in k or k == 'opening_hours':
      tag_k_v_yes_no_auditing.add(k)
  return tag_k_v_yes_no_auditing

```

#### 6- Auditoria de sub-elementos e ajustes para normalização de dados

##### 6.1- Normalização de zipcode (CEP)

Conforme analisado os dados foram encontradas algumas inconsistências que serão ajustadas:

1- Códigos de código postal com formato incorreto ou fora do range para new york, conforme a wikipedia o prefixo dos códigos postais de new york vão de 100 a 149 e o código postal deve conter 5 dígitos:

[https://en.wikipedia.org/wiki/List_of_ZIP_code_prefixes] (https://en.wikipedia.org/wiki/List_of_ZIP_code_prefixes)

Conforme o codigo de auditoria abaixo foi identificado alguns zipcodes com inconsistencia:

```python
def audit_postal_code(postal_code, element):
  if element.tag == 'tag' and element.attrib['k'] in ['addr:zip', 'addr:postcode']:
    if len(element.attrib['v']) != 5:
      postal_code.add(element.attrib['v'])
    else:
      try:
        v = int(element.attrib['v'])
        if not (POSTAL_CODE_NY_RANGE[0] <= v <= POSTAL_CODE_NY_RANGE[1]):
          postal_code.add(element.attrib['v'])
      except ValueError: 
        postal_code.add(element.attrib['v'])
    
  return postal_code
```

Observações sobre as inconsistencias de zipcode possíveis processos de limpeza:

- Os códigos com prefixo 070, 073, 076, etc ..., são de New Jersey proximo a New York, devido ao quadrante que foi pego como referencia, devido a isso não será alterado;
- Os códigos com 4 dígitos após o '-' e os 5 dígitos, para resolver deverá ser utilizado apenas os primeiros 5 dígitos antes do '-'
- O código postal 83 por exemplo, é referente ao Central Park, devido a isso não será alterado:
```xml
    <tag k="addr:housenumber" v="34" />
    <tag k="addr:postcode" v="83" />
    <tag k="addr:street" v="Central Park North" />
```
- Os códigos que possuem o prefixo NY ou o texto New York, NY , será removido e deixado apenas o zipcode.

Para normalizar o zipcode foi utilizado o seguinte código:

```python

    def normalize_and_clean_zip_code(zipcode):
      zip = zipcode.split()
      for z in zip:
        if z.isdigit() or len(z) == 10 or len(z) == 11 or (len(z) == 5 and z.isdigit()):
          zipcode = z
          break

      return zipcode
```

##### 6.2- Normalização de Inconsistências em nomes de endereço:

- Para identificar as inconsistencias em nomes de endereços foi realizado um split por espaços no nome do endereço e colocado o nome em uppercase, inserido e um dicionárioe contabilizado a quantidade de ocorrências para avaliar o impácto nos dados, e observado as ocorrencias diretamente no xml osm para garantir os dados, os principais erros encontrados foram em grandes abreviações em nomes de endereço, como exemplos abaixo:
```xml
    <tag k="addr:street" v="W. 44th street"/>
    <tag k="addr:street" v="E 55th St Ste. 301"/>
    <tag k="addr:street" v="Prince st."/>
```

O código para avaliar o impacto e gerar o dicionário:

```python
def audit_street_name(street_address, element):
  if element.tag == 'tag' and element.attrib['k'] == 'addr:street':
    v = element.attrib['v'].upper().split(' ')
    for valor in v:
      count = 1
      if valor in street_address:
        count = street_address[valor] + 1
      street_address[valor] = count
  return street_address

```

Os principais casos de erro e abreviação no endereço, abaixo foram pegos os dados conforme análise dos logs:

- Pontos Cardinais
'WEST' : 21192 ,
'W.' : 3 ,
'W' : 20 ,

'SOUTH' : 2056 ,
'S' : 22 ,

'NORTH' : 1369 ,
'N' : 2 ,

'EAST' : 18103 ,
'E.' : 2 ,
'E' : 17 ,

- Tipos de Endreço:

'ROAD' : 1701 ,
'RD' : 1 ,

'STREET' : 98465 ,

'STREEET' : 1 ,
'STEET' : 2 ,
'ST.,' : 3 ,
'ST.' : 21 ,
'ST,' : 1 ,
'ST' : 546 ,
'STREER' : 1 ,

'SUITE' : 6 ,
'STE.' : 1 ,

'PLACE' : 3786 ,
'PL' : 1 ,

'BOULEVARD' : 3286 ,

'BLVD' : 8 ,
'BLV.' : 1 ,

'AVENUE' : 49407 ,

'AVENUE;DEKALB' : 1 ,
'AVENUE,#392' : 5 ,
'AVENUE,' : 2 ,

'AVENEU' : 1 ,
'AVE.' : 1 ,
'AVE,' : 2 ,
'AVE' : 105 ,

Tipos de endereço esperados:

["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", "Trail", "Parkway", "Commons"]

Obs.: caso o nome da rua inicie com St. deverá continuar como st e não ser alterado, como por exemplo.
    
    <tag k="addr:street" v="St. Nicholas Aveneu"/>
    
Obs.2: Alterações de pontos cardinais como East, North, West e South deverão ser alterados apenas quando a palavra a ser corrigida esteja no inicio do endereço. 

Para realizar as correções criei as variáveis abaixo com as informações de substituição.

FIX_STREET_TYPE = { 
  "RD": "ROAD",
  "RD.": "ROAD",
  "STREEET" : "STREET",
  "STEET" : "STREET",
  "ST.," : "STREET",
  "ST." : "STREET",
  "ST," : "STREET",
  "ST" : "STREET",
  "STREER" : "STREET",
  "STE" : "SUITE",
  "STE." : "SUITE",
  "STE," : "SUITE",
  "PL" : "PLACE",
  "BLVD" : "BOULEVARD",
  "BLV." : "BOULEVARD",
  "BLV," : "BOULEVARD",
  "AVENUE," : "AVENUE",
  "AVENEU" : "AVENUE",
  "AVE." : "AVENUE",
  "AVE," : "AVENUE",
  "AVE" : "AVENUE",
}

FIX_CARDINAL_NAMES = {
  'W.' : 'WEST',
  'W'  : 'WEST',
  'S'  : 'SOUTH',
  'N'  : 'NORTH',
  'E.' : 'EAST',
  'E'  : 'EAST'
}


Para normalização do nome do endereço foi utilizado o seguinte códito:

```python

    def normalize_and_clean_street_name(street_address_name):
      street_name_normalized = []
      for i, s in enumerate(street_address_name.upper().split()):
        s_new = s
        if s not in EXPRECTED_STREET_TYPE:
          if i > 0 and s in FIX_STREET_TYPE:
            s_new = FIX_STREET_TYPE[s]

        if s in FIX_CARDINAL_NAMES:
          s_new = FIX_CARDINAL_NAMES[s]

        street_name_normalized.append(s_new)
      s = " ".join(street_name_normalized)
      pprint.pprint(s)

      return s
```
##### 6.3- Normalização de Inconsistências em nomes da tag com a k="name":

- Nomes com `'`que podem quebrar o json no momento do processamento
- Nomes com `/`
- Nomes com `-`

Correções em nomes para normalização:
- Nomes que conterem os caracteres `" ' | \ / -` irei colocar um espaço em branco no local
- Converter todos os nomes para uppercase



```xml
    <tag k="name" v="McSorley's Old Ale House" />

    <tag k="name" v="CVS/pharmacy"/>

    <tag k="name" v="Rite-Aid"/>
```


## Visão Geral dos Dados

#### As estatísticas das informações gerais sobre o conjunto de dados foram computadas?

Consultas de banco de dados são usados para fornecer uma visão estatística do conjunto de dados, como:

###### Tamanho do arquivo

### File sizes

```
map.osm ............... 372,5 MB
map.osm.json .......... 423,1 MB

Total de registros processados: ~ 1.6 M

Estatistica do banco de dados (MongoDB):

> show databases
2ia                               0.000GB
admin                             0.000GB
config                            0.000GB
local                             0.000GB
test                              0.003GB
udacity_datascience_for_business  0.138GB

> show collections
node

> db.node.count()
1605637

```

###### Número de usuários únicos

```
db.node.distinct( 'created.uid' ).length

2707
```

###### Número de nós e caminhos

``` code
db.node.aggregate(
   [
      {
        $group : {
           _id : '$type',
           count:  { $sum: 1 }
        }
      }
   ]
)

{ "_id" : "way", "count" : 230055 }
{ "_id" : "node", "count" : 1375582 }
```

###### Número de tipo de nós escolhidos Farmacias, Restaurantes e FastFoods.

```
db.node.aggregate(
   [
    { $match: { 'primary_map_feature.amenity' : { $in : ['pharmacy', 'restaurant', 'fast_food'] } } },
      {
        $group : {
           _id : '$primary_map_feature.amenity',
           count:  { $sum: 1 },
        }
      },
      { $sort: { count : -1 } }
   ]
)

{ "_id" : "restaurant", "count" : 2854 }
{ "_id" : "fast_food", "count" : 672 }
{ "_id" : "pharmacy", "count" : 295 }
```

###### Estatísticas adicionais que não estão na lista acima:

- Top 10 restaurantes com mais lojas:

```
db.node.aggregate(
   [
    { $match: {
       $and : [ {'primary_map_feature.amenity' : 'restaurant' } , {'name' : {$ne : null} }] } },
      {
        $group : {
           _id : '$name',
           count:  { $sum: 1 },
        }
      },
      { $sort: { count : -1 } },
      {$limit : 10}
   ]
)

{ "_id" : "BAREBURGER", "count" : 13 }
{ "_id" : "CHIPOTLE", "count" : 8 }
{ "_id" : "BLOCKHEADS", "count" : 5 }
{ "_id" : "XI`AN FAMOUS FOODS", "count" : 5 }
{ "_id" : "SPICE", "count" : 5 }
{ "_id" : "SERAFINA", "count" : 5 }
{ "_id" : "LE PAIN QUOTIDIEN", "count" : 4 }
{ "_id" : "DALLAS BBQ", "count" : 4 }
{ "_id" : "SWEETGREEN", "count" : 4 }
{ "_id" : "BILL`S BAR & BURGER", "count" : 4 }
```


- Top 10 fast foods com mais lojas:

```
db.node.aggregate(
   [
    { $match: {
       $and : [ {'primary_map_feature.amenity' : 'fast_food' } , {'name' : {$ne : null} }] } },
      {
        $group : {
           _id : '$name',
           count:  { $sum: 1 },
        }
      },
      { $sort: { count : -1 } },
      {$limit : 10}
   ]

)

{ "_id" : "MCDONALD`S", "count" : 70 }
{ "_id" : "SUBWAY", "count" : 45 }
{ "_id" : "DUNKIN` DONUTS", "count" : 20 }
{ "_id" : "CHIPOTLE", "count" : 18 }
{ "_id" : "CHIPOTLE MEXICAN GRILL", "count" : 12 }
{ "_id" : "TACO BELL", "count" : 11 }
{ "_id" : "BURGER KING", "count" : 9 }
{ "_id" : "FIVE GUYS", "count" : 9 }
{ "_id" : "DUNKIN DONUTS", "count" : 9 }
{ "_id" : "SHAKE SHACK", "count" : 9 }
```

- Top 10 farmacias com mais lojas:

```
db.node.aggregate(
   [
    { $match: {
       $and : [ {'primary_map_feature.amenity' : 'pharmacy' } , {'name' : {$ne : null} }] } },
      {
        $group : {
           _id : '$name',
           count:  { $sum: 1 },
        }
      },
      { $sort: { count : -1 } },
      {$limit : 10}
   ]
)

{ "_id" : "DUANE READE", "count" : 89 }
{ "_id" : "RITE AID", "count" : 36 }
{ "_id" : "CVS", "count" : 24 }
{ "_id" : "CVS PHARMACY", "count" : 18 }
{ "_id" : "WALGREENS", "count" : 9 }
{ "_id" : "DUANE READE PHARMACY", "count" : 2 }
{ "_id" : "RITE AIDE", "count" : 2 }
{ "_id" : "DUANEREADE", "count" : 2 }
{ "_id" : "CITY CHEMIST", "count" : 2 }
{ "_id" : "STEINWAY STREET PHARMACY", "count" : 1 }
```

- Top 10 farmacias com mais lojas que tem funcionamento nos domingos:

``` code
db.node.aggregate(
   [
    { $match: {
       $and : [{'restrictions_rules.opening_hours.yes' : {$exists : 'sunday'}},  {'primary_map_feature.amenity' : pharmacy' } , {'name' : {$ne : null} }] } },
      {
        $group : {
           _id : '$name',
           count:  { $sum: 1 },
        }
      },
      { $sort: { count : -1 } },
      {$limit : 10}
   ]

)

{ "_id" : "DUANE READE", "count" : 50 }
{ "_id" : "RITE AID", "count" : 21 }
{ "_id" : "CVS PHARMACY", "count" : 15 }
{ "_id" : "CVS", "count" : 7 }
{ "_id" : "WALGREENS", "count" : 3 }
{ "_id" : "DUANE READE PHARMACY", "count" : 2 }
{ "_id" : "GST PHARMACY", "count" : 1 }
{ "_id" : "VERNON BLVD PHARMACY", "count" : 1 }
{ "_id" : "92 PHARMACY INC.", "count" : 1 }
{ "_id" : "CHELSEA ROYAL CARE PHARMACY", "count" : 1 }
```

## Outras Ideias sobre os Dados

- Uma ideia adicional seria os orgão do governo ou ler de outros datasets publicos auxiliarem no enriquecimento dos dados, até mesmo os próprios estabelecimentos enviarem dados.

Por Exemplo, utilizando os dados de farmacia:

    No total de 295 farmacias apenas 94,23% possuem nome registrado e apenas 61,69% possuem nome do estabelecimento e endereço registrados.
    
```
> db.node.find({'primary_map_feature.amenity' : 'pharmacy'}).length()
295

> db.node.find( {$and : [ {'primary_map_feature.amenity' : 'pharmacy'}, {name : { $ne : null} } ]}).length()

278

> db.node.find( {$and : [ {'primary_map_feature.amenity' : 'pharmacy'}, {name : { $ne : null} }, { address: { $exists: true } } ]}).length()

182
```

Sendo assim, para ter uma base mais consisa teria que ter muito mais divulgação e auxilio de todas as comunidades para melhorar os dados e consistencia destes dados, pois não adianta ter uma base tão grande se não for atualizada.


# Conclusão

Este mapa se refere ao quadrante referente a Manhanttan em New York City, e também às areas ao redor devido ao quadrante escolhido. Como conclusão do trabalho ficou entendido que os dados estudados foram muito bons para o entendimento, prática e o aprendizado na limpeza dos dados e estruturação de objetos, assim como melhorar sua análise. Aqui vale uma observação para esta conclusão: segundo os dados utilizados, a massa de dados é bem grande, porém existem inconsistencias e ajustes a serem feitos.
É interessante notar que alguns problemas foram encontrados durante a fase de análise dos dados e que os dados obtidos refere-se ao translado de usuários que frequentemente utilizaram o Taxi como meio de transporte.

-- Solução: Auditoria de tags e estruturação do json
-  Benefício: Limpeza dos dados para chaves inválidas

-- Solução: Auditoria de sub-elementos e ajustes para normalização de dados
Benefícios

-  Benefício 1: Normalização dos zipcode (CEP)
-  Benefício 2: Normalização de Inconsistências em nomes de endereço
-  Benefício 3: Normalização de Inconsistências em nomes de Tags 

-- Problemas esperados
- Problema 1: Normalização do zipcode
- Problema 2: Normalização do nome do endereço
